In [1]:
from sklearn.datasets import fetch_openml

In [2]:
mnist = fetch_openml("mnist_784", version=1, cache=True, as_frame=False)

In [3]:
X = mnist.data
y = mnist.target

In [4]:
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=1/7, random_state=0)
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.LongTensor(list(map(int, y_train)))
y_test = torch.LongTensor(list(map(int, y_test)))

In [6]:
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable

In [7]:
X_train = X_train.view(-1, 1, 28,28).float()
X_test = X_test.view(-1, 1, 28,28).float()
print(X_train.shape)
print(X_test.shape)

torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])


In [8]:
train = TensorDataset(X_train, y_train)
test = TensorDataset(X_test, y_test)
BATCH_SIZE = 32
loader_train = DataLoader(train, batch_size= BATCH_SIZE, shuffle=False)
loader_test = DataLoader(test, batch_size= BATCH_SIZE, shuffle=False)


In [14]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1,32,kernel_size=5)
        self.conv2 = nn.Conv2d(32,32,kernel_size=5)
        self.conv3 = nn.Conv2d(32,64,kernel_size=5)
        self.fc1 = nn.Linear(3 * 3 * 64, 256)
        self.fc2 = nn.Linear(256, 10)

        self.loss_fn = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters(), lr=0.01)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1,3*3*64)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

    

In [15]:
def fit(model, loader_train):
    optimizer = torch.optim.Adam(model.parameters())
    error = nn.CrossEntropyLoss()
    EPOCHS = 1
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(loader_train):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()
            predicted = torch.max(output.data, 1)[1]
            correct += (predicted == var_y_batch).sum()
            if batch_idx % 50 == 0:
                print('에포크 : {} [{}/{} ({:.0f}%)]\t 손실함수 : {:.6f}\t Accuracy:{:.3f}%'\
                      .format(epoch, batch_idx*len(X_batch),len(loader_train),\
                        100.*batch_idx / len(loader_train),loss.data,correct*100./ (BATCH_SIZE*(batch_idx + 1))))


In [16]:
def evaluate(model):
    correct = 0
    for test_imgs, test_labels in loader_test:
        test_imgs = Variable(test_imgs).float()
        output = model(test_imgs)
        predicted = torch.max(output,1)[1]
        correct += (predicted == test_labels).sum()
    print("테스트 데이터 정확도: {:.3f}% ".format(float(correct) / (len(loader_test)*BATCH_SIZE)))

cnn = CNN()
evaluate(cnn)
fit(cnn, loader_train)
cnn.eval()
evaluate(cnn)
index = 10
data = X_test[index].view(-1, 1, 28, 28).float()
output = cnn(data)
print('{} 번째 학습데이터의 테스트 결과 : {}'.format(index,output))
_, predicted = torch.max(output, 1)
print('{} 번째 데이터의 예측측 : {}'.format(index, predicted.numpy()))
print('실제 레이블 : {}'.format(y_test[index]))

테스트 데이터 정확도: 0.107% 
에포크 : 0 [0/1875 (0%)]	 손실함수 : 21.897602	 Accuracy:12.500%
에포크 : 0 [1600/1875 (3%)]	 손실함수 : 1.887379	 Accuracy:15.748%
에포크 : 0 [3200/1875 (5%)]	 손실함수 : 1.083102	 Accuracy:32.673%
에포크 : 0 [4800/1875 (8%)]	 손실함수 : 0.997207	 Accuracy:44.454%
에포크 : 0 [6400/1875 (11%)]	 손실함수 : 0.651295	 Accuracy:51.757%
에포크 : 0 [8000/1875 (13%)]	 손실함수 : 1.057168	 Accuracy:57.234%
에포크 : 0 [9600/1875 (16%)]	 손실함수 : 0.426731	 Accuracy:61.171%
에포크 : 0 [11200/1875 (19%)]	 손실함수 : 0.432984	 Accuracy:64.459%
에포크 : 0 [12800/1875 (21%)]	 손실함수 : 0.427730	 Accuracy:67.106%
에포크 : 0 [14400/1875 (24%)]	 손실함수 : 0.529925	 Accuracy:69.090%
에포크 : 0 [16000/1875 (27%)]	 손실함수 : 0.704135	 Accuracy:70.752%
에포크 : 0 [17600/1875 (29%)]	 손실함수 : 0.290246	 Accuracy:72.238%
에포크 : 0 [19200/1875 (32%)]	 손실함수 : 0.722096	 Accuracy:73.591%
에포크 : 0 [20800/1875 (35%)]	 손실함수 : 0.473601	 Accuracy:74.712%
에포크 : 0 [22400/1875 (37%)]	 손실함수 : 0.183348	 Accuracy:75.678%
에포크 : 0 [24000/1875 (40%)]	 손실함수 : 0.701817	 Accuracy:76.481%
